# NOTE:
You will need to have installed the Google Cloud SDK and a project set up to run this work flow interactively. Instructions on how to set up the Google Cloud SDK are [here](https://cloud.google.com/sdk/docs/install).

In [1]:
# pip install --upgrade nbformat


In [2]:
# from io import StringIO
import ee


In [3]:
ee.Initialize()

In [4]:
# from gee_water import utils
# from gee_water.utils import mask

In [5]:
# import gee_water
from gee_water.utils import (
    mask_hls_l30_cloud_shadow,
    add_ndvi_l30,
    create_monthly_composites,
    get_time_series
)

# README
The purpose of this notebook is to do some exploratory work on NDVI for regions in Southern Baja for a blog post exploring water resources in the area.
The focus region for this project will be Baja California Sur, near the southern most tip of the peninsula in the La Paz - Cabo - Todos Santos triangle.

## Define Region of Interest

In [6]:
# Coordinates for La Paz, BCS
la_paz_coords = ee.Geometry.Point([-110.3166667, 24.1422222])
buffer_distance_meters = 75 * 1609.34  # 75 miles in meters
roi = la_paz_coords.buffer(buffer_distance_meters)

# Run NDVI Analysis Using HLS GEE Data Set

In [7]:
# For this particular blog, I am analyzing multiple datasets from a handful of different 
# satellites. The year 2013 is far enough back historically to capture form trends, but still 
# allows for higih quality data to be gathred for all my metrics of interest.
start_date = '2013-01-01'
end_date   = '2023-12-31'


In [8]:
hls_l30_with_ndvi = (
    ee.ImageCollection('NASA/HLS/HLSL30/v002')
    .filterDate(start_date, end_date)
    .filterBounds(roi)
    .map(mask_hls_l30_cloud_shadow)
    .map(add_ndvi_l30)
)


In [9]:
# create composites
hls_l30_ndvi_monthly = create_monthly_composites(hls_l30_with_ndvi)

In [10]:
# create time series data from the monthly composites
hls_l30_ndvi_timeseries = get_time_series(hls_l30_ndvi_monthly, 'NDVI', roi)

In [11]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import timedelta

In [24]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

def plot_multiple_timeseries_with_trendlines_plotly(
    df_list,
    date_col='date',
    value_col='value',
    labels=None,
    x_interval_months=6,
    title='Time Series Comparison',
    ylabel='Value (units)'
):
    """
    Plots multiple time series (date vs. value) with linear trend lines in a single Plotly figure.
    Uses "days since earliest global date" as the x-values for linear regression, 
    which avoids extremely large timestamp numbers in nanoseconds.

    Args:
        df_list (list of pd.DataFrame):
            A list of DataFrames, each with a date column and a value column.
        date_col (str):
            Name of the column containing dates in each DataFrame.
        value_col (str):
            Name of the column containing the value to be plotted.
        labels (list of str or None):
            Legend labels for each DataFrame. If None, uses generic labels.
        x_interval_months (int):
            Interval (in months) for major x-axis ticks. Default=6 => tick every 6 months.
        title (str):
            Title of the plot.
        ylabel (str):
            Y-axis label.

    Returns:
        plotly.graph_objects.Figure:
            A Plotly figure object that can be displayed using fig.show().
    """

    # If no labels are given, assign default ones
    if labels is None:
        labels = [f"Dataset {i}" for i in range(len(df_list))]
    elif len(labels) != len(df_list):
        raise ValueError("Length of `labels` must match length of `df_list`.")

    # 1. Find a global earliest date among all DataFrames
    global_earliest_date = min(df[date_col].min() for df in df_list)

    # Create a Plotly figure
    fig = go.Figure()

    # 2. Generate each time series + trend lines
    for i, df in enumerate(df_list):
        # Sort by date just in case
        df_sorted = df.sort_values(by=date_col).reset_index(drop=True)

        # Calculate "days since global earliest date" for slope-fitting
        df_sorted['days_since_start'] = (
            df_sorted[date_col] - global_earliest_date
        ).dt.days

        # x-values for plotting (actual dates) vs. x-values for slope-fitting (days_since_start)
        x_dates = df_sorted[date_col]
        x_days = df_sorted['days_since_start']
        y_vals = df_sorted[value_col]

        # Plot the main time series
        fig.add_trace(
            go.Scatter(
                x=x_dates,
                y=y_vals,
                mode='lines',
                name=labels[i]
            )
        )

        # Compute trend (slope) per day via np.polyfit
        coeffs = np.polyfit(x_days, y_vals, 1)  # linear fit
        slope_per_day, intercept = coeffs[0], coeffs[1]

        # Evaluate the linear model at each x_days
        trend_line_y = intercept + slope_per_day * x_days

        # Convert slope to "units per year"
        slope_per_year = slope_per_day * 365.25

        # Plot the trend line
        fig.add_trace(
            go.Scatter(
                x=x_dates,  # still plot trend line against real dates
                y=trend_line_y,
                mode='lines',
                line=dict(dash='dash'),
                name=f"{labels[i]} Trend (slope={slope_per_year:.5f}/yr)"
            )
        )

    # 3. Format the x-axis ticks and other layout details
    # Tick every x_interval_months months
    dtick_str = f"M{x_interval_months}"

    fig.update_layout(
        title=title,
        xaxis=dict(
            title='Date',
            dtick=dtick_str,
            tickformat='%Y-%m'
        ),
        yaxis=dict(
            title=ylabel
        ),
        legend=dict(
            x=0.01,
            y=0.99
        ),
        margin=dict(l=60, r=40, t=60, b=60)
    )

    return fig


In [26]:
plot_multiple_timeseries_with_trendlines_plotly(
    df_list=[hls_l30_ndvi_timeseries],
    date_col='date',
    value_col='NDVI',
    labels=['HLS NDVI'],
    x_interval_months=6,
    title='HLS Derived NDVI 2013 - 2023',
    ylabel='Value (units)'
)

# Determine if Trend is Statistically Significant

# Display NDVI Year to Year

In [97]:
def monthly_ndvi_image(collection, band, year, month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')

    return (collection
            .filterDate(start_date, end_date)
            # Additional filtering for clouds, if you like
            .select(band)
            .mean()
            .rename(f'{band}_{year}_{month}'))

In [98]:
# June early NDVI
hls_ndvi_june_2013 = monthly_ndvi_image(hls_l30_with_ndvi, 'NDVI', 2013, 6)

# June 2023 NDVI
hls_ndvi_june_2023 = monthly_ndvi_image(hls_l30_with_ndvi, 'NDVI', 2023, 6)

# December early NDVI
hls_ndvi_dec_2013 = monthly_ndvi_image(hls_l30_with_ndvi, 'NDVI', 2013, 12)

# December 2023 NDVI
hls_ndvi_dec_2023 = monthly_ndvi_image(hls_l30_with_ndvi, 'NDVI', 2023, 12)


In [99]:
ndvi_vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['brown', 'white', 'green']
}

In [131]:
Map = geemap.Map(center=[24.1422222, -110.3166667], zoom=12)

# Create Tile Layers for 2013 and 2023 VIIRS Composites
left_layer_viirs = geemap.ee_tile_layer(hls_ndvi_june_2013, ndvi_vis_params, 'VIIRS 2013')
right_layer_viirs = geemap.ee_tile_layer(hls_ndvi_june_2023, ndvi_vis_params, 'VIIRS 2023')

# Add a split map feature
Map.split_map(left_layer=left_layer_viirs, right_layer=right_layer_viirs)

# Display the map
Map

In [132]:
# Map.to_html('./../static/html/ndvi_test.html')

In [ ]:
def create_split_viirs_map(output_html, coords, left_layer, right_layer, vis_params):
    """
    Creates a split map with VIIRS composites for 2013 and 2023, adds a legend, 
    and exports the map as an HTML file for embedding.

    Parameters:
    - output_html (str): Path to save the output HTML file.
    - coords (dict): Dictionary with 'coordinates' key for center of the map [lon, lat].
    - viirs_2013_id (str): Earth Engine ID for the 2013 VIIRS composite.
    - viirs_2023_id (str): Earth Engine ID for the 2023 VIIRS composite.
    - vis_params (dict): Visualization parameters for the VIIRS layers.

    Returns:
    - None
    """
    # Initialize the Earth Engine API
    ee.Initialize()

    # Load VIIRS images
    viirs_2013 = ee.Image(viirs_2013_id)
    viirs_2023 = ee.Image(viirs_2023_id)

    # Create an interactive map centered on the specified coordinates
    Map = geemap.Map(center=coords['coordinates'][::-1], zoom=12)

    # Create Tile Layers for 2013 and 2023 VIIRS Composites
    left_layer_viirs = geemap.ee_tile_layer(viirs_2013, vis_params, 'VIIRS 2013')
    right_layer_viirs = geemap.ee_tile_layer(viirs_2023, vis_params, 'VIIRS 2023')

    # Add split map feature
    Map.split_map(left_layer=left_layer_viirs, right_layer=right_layer_viirs)

    # Add a legend to the map
    legend_dict = {
        'Low': 'blue',
        'Medium': 'green',
        'High': 'red'
    }
    Map.add_legend(legend_title="VIIRS Intensity", legend_dict=legend_dict)
